In [55]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [56]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root = './data', train = True,
                                        download = True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size,
                                          shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root='./data', train = False,
                                       download = True, transform = transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size,
                                         shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Files already downloaded and verified
Files already downloaded and verified


In [57]:
net = nn.Sequential(
    nn.Conv2d(3, 64, 5),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),

    nn.Conv2d(64, 64, 1),
    nn.Conv2d(64, 64, 3),
    nn.Conv2d(64, 256, 1),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.Conv2d(256, 128, 1),
    nn.Conv2d(128, 128, 3),
    nn.Conv2d(128, 512, 1),
    nn.BatchNorm2d(512),
    nn.ReLU(),

    nn.Conv2d(512, 256, 1),
    nn.Conv2d(256, 256, 3),
    nn.Conv2d(256, 1024, 1),
    nn.BatchNorm2d(1024),
    nn.ReLU(),

    nn.Conv2d(1024, 512, 1),
    nn.Conv2d(512, 512, 3),
    nn.Conv2d(512, 2048, 1),
    nn.BatchNorm2d(2048),
    nn.ReLU(),

    nn.Conv2d(2048, 1024, 1),
    nn.Conv2d(1024, 1024, 3),
    nn.Conv2d(1024, 4096, 1),
    nn.BatchNorm2d(4096),
    nn.ReLU(),

    nn.Conv2d(4096, 2048, 1),
    nn.Conv2d(2048, 2048, 3),
    nn.Conv2d(2048, 8192, 1),
    nn.BatchNorm2d(8192),

    nn.AvgPool2d(2),
    nn.Flatten(),

    nn.Linear(8192, 120),
    nn.ReLU(),
    nn.Linear(120, 84),
    nn.ReLU(),
    nn.Linear(84, 10)
)

net.to(device)

In [58]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:

        images, labels = data.to(device)
        outputs = net(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 53 %
